In [16]:
import ollama
import sqlite3
import pandas as pd
import re

from tqdm.notebook import tqdm

In [2]:
def format_test_example(row) -> str:
    return f"""<|system|>
Ты — интеллектуальный текстовый помощник, преобразующий команды пользователя в корректные SQL-запросы на языке SQLite.

Ты всегда работаешь с таблицей `transactions`, имеющей следующую структуру:

- id (INTEGER, PRIMARY KEY)
- user_id (TEXT)
- type (TEXT: 'income' или 'expense')
- category (TEXT)
- amount (REAL)
- date (TIMESTAMP, по умолчанию date('now', 'localtime'))

Описание:
- `income` — это доход
- `expense` — это трата

Твоя задача — по команде пользователя с его ID сгенерировать корректный SQL-запрос к этой таблице.

Важно:
- Используй только SQLite-синтаксис, без пояснений.
- Не пиши комментарии.
- Не используй несуществующие поля (например, `note`).
- Поле `date` можно указывать через date('now', '-N day') или опустить (будет по умолчанию).

Пример:
---
Пользователь с ID user_1 дал команду:
"Добавь трату 500 рублей на еду вчера"

Ответ:
INSERT INTO transactions (user_id, type, category, amount, date)
VALUES ('user_1', 'expense', 'еда', 500, date('now', '-1 day'));
---
</s>
<|user|>
Пользователь с ID {row["user_id"]} дал команду:
"{row["user_command"]}"
</s>
<|assistant|>
"""

In [4]:
def clean_sql(text):
    """
    Удаляет обёртки ```sql ... ``` и специальные токены вроде </s>, <|...|>.
    """
    if not isinstance(text, str):
        return ""

    # Убираем блоки ```sql и ```
    text = re.sub(r"```sql\s*", "", text, flags=re.IGNORECASE)
    text = re.sub(r"```", "", text)

    # Убираем спецтокены LLM
    text = re.sub(r"</s>", "", text)
    text = re.sub(r"<\|.*?\|>", "", text)

    return text.strip()

In [10]:
def generate_sql_from_df(test_df, client, model):
    predicted_sqls = []

    for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
        prompt = format_test_example(row)
        response = client.generate(model=model, prompt=prompt)

        sql = response.response.split("<|assistant|>")[-1].strip()
        predicted_sqls.append(clean_sql(sql))

    test_df["predicted_sql"] = predicted_sqls
    return test_df

In [54]:
def normalize_sql(sql: str) -> str:
    return sql.strip().lower().rstrip(";")

def exact_match(pred: str, gold: str) -> bool:
    return normalize_sql(pred) == normalize_sql(gold)

def execute_sql(sql: str, db_path: str):
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()
        cursor.execute(sql)
        rows = cursor.fetchall()
        conn.close()
        return sorted(rows)
    except Exception as e:
        return f"[ERROR] {e}"

In [55]:
def evaluate_sql_quality(df, sql_col="predicted_sql", gold_col="gold_sql", db_path="db_path"):
    exact_matches = []
    execution_matches = []

    for _, row in df.iterrows():
        pred_sql = row[sql_col]
        gold_sql = row[gold_col]

        exact = exact_match(pred_sql, gold_sql)
        exact_matches.append(exact)

        pred_result = execute_sql(pred_sql, db_path)
        gold_result = execute_sql(gold_sql, db_path)
        exec_match = pred_result == gold_result
        execution_matches.append(exec_match)

    df["exact_match"] = exact_matches
    df["exec_match"] = execution_matches

    exact_acc = sum(exact_matches) / len(df)
    exec_acc = sum(execution_matches) / len(df)

    return {
        "exact_match_accuracy": round(exact_acc, 4),
        "execution_accuracy": round(exec_acc, 4),
        "df": df
    }

In [71]:
client = ollama.Client()

In [12]:
test_df = pd.read_csv("data/fin_ass_test.csv")

In [13]:
test_df.shape

(1000, 3)

In [72]:
models = ["llama3.2:1b", "llama3.2:3b", "gemma3:4b", "gemma3:1b"]

In [63]:
all_results = {
    "model": [],
    "exact_match_accuracy": [],
    "execution_accuracy": []
}

In [64]:
for model in models:
    print(f"Execution for {model} model...")
    temp = generate_sql_from_df(test_df=test_df,client=client,model=model)
    results = evaluate_sql_quality(temp, db_path="data/transactions.db")
    all_results["model"].append(model)
    all_results["exact_match_accuracy"].append(results["exact_match_accuracy"])
    all_results["execution_accuracy"].append(results["execution_accuracy"])

Execution for llama3.2:1b model...


  0%|          | 0/1000 [00:00<?, ?it/s]

Execution for llama3.2:3b model...


  0%|          | 0/1000 [00:00<?, ?it/s]

Execution for gemma3:4b model...


  0%|          | 0/1000 [00:00<?, ?it/s]

In [70]:
pd.DataFrame(all_results)

,model,exact_match_accuracy,execution_accuracy
0,llama3.2:1b,0.001,0.274
1,llama3.2:3b,0.066,0.547
2,gemma3:4b,0.150,0.965
3,gemma3:1b,0.029,0.533


In [74]:
qwen_models = ["qwen3:1.7b", "qwen3:4b"]

In [75]:
for model in qwen_models:
    print(f"Execution for {model} model...")
    temp = generate_sql_from_df(test_df=test_df,client=client,model=model)
    results = evaluate_sql_quality(temp, db_path="data/transactions.db")
    all_results["model"].append(model)
    all_results["exact_match_accuracy"].append(results["exact_match_accuracy"])
    all_results["execution_accuracy"].append(results["execution_accuracy"])

Execution for qwen3:1.7b model...


  0%|          | 0/1000 [00:00<?, ?it/s]

KeyboardInterrupt: 